# Todo List

- Create compact trie, to save space
    - 5/21 CURRENT PROGRESS: compact trie insert is correct! now work on search
    - Small change: to save space, maybe don't create a Node at every array index? Becuase we might overwrite it anyways
- Next array should only have intervals for two octaves up or down. If there is an interval stored that is larger than that, make it a linked list idea
    - So the main four octave region is like one big node, and then there are nodes added before or after as needed if larger intervals are required
- TODOs in code:
    - terminal value may need to be array
    - in search, return result before updating mostSearched. **Alternatively**, only update mostSearched if the returned musical work is confirmed by the user to be the work they were looking for.
- BIG MILESTONE: combine with the html pianokeyboard
    - For flat vs. sharp, maybe use alt/option to toggle: https://stackoverflow.com/questions/13539493/how-to-detect-keyboard-modifier-ctrl-or-shift-through-javascript

---
    
- Further features
    - Musical autocorrect
        - Highlight intervals that may not be correct, by:
            1. comparing to similar melodies in the trie
            2. checking hard-coded intervals/sequences that are infrequent or sound very dissonant
        - Octave differences. If user enters in `E4 C4` instead of `E4 C5`, be able to still possibly find the correct musical work

# Imports and Test Cases

In [1]:
import sys
import os

sys.path.append("../")

In [2]:
from trie.trie import *

# for testing
import tracemalloc
from itertools import permutations
import random

In [3]:
# Define some pieces/songs
works = []

works.append(ClassicalPiece('Fur Elise', 'Ludwig van Beethoven', 1810, 'Classical Period', 'a minor',
                            'E5 D#5 E5 D#5 E5 B4 D5 C5 A4 / C4 E4 A4 B4 / E4 G#4 B4 C5',
                            None, None))

works.append(Song('Happy Birthday!', None, None, None, None, 'C4 C4 D4 C4 F4 E4'))
works.append(Song('Twinkle Twinkle Little Star', None, None, None, None, 'C4 C4 G4 G4 A4 A4 G4'))

works.append(Song('Avatar\'s Love', None, 2003, 'TV Show Music', 'C major', 'C5 B4 G4 E4'))
works.append(Song('Jurrasic Park Theme', 'John Williams', 1993, 'Movie Score', 'C major',
                  'C5 B4 C5 G4 F4 C5 B4 C5 G4 F4 C5 B4 C5 D5 D5 F5 F5 / E5 C5 D5 B4 G4 E5 C5 D5 / ' + 
                  'G5 C5 F5 E5 E5 D5 D5'))
# works.append(Song('Jurrasic Park Theme', 'John Williams', 1993, 'Movie Score', 'C major',
#                   'C5 B4 C5 G4 F4 C5 B4 C5 G4 F4 C5 B4 C5 D5 D5 F5 F5'))
works.append(Song('Le Festin', 'Ratatouille', 2000, 'Movie Score', 'Bb major',
                  'Bb3 G4 F4 Eb4 G4 F4 Eb4 G4 F4 Eb4 Bb4'))
works.append(Song('Jazz Lick', None, None, 'Jazz', 'C major', 'C3 E2 F2 F#2 G2 A2 B2 C2'))

works.append(MusicalWork('custom', 'bruh', 2021, 'yerp', 'C major', 'C5 B4 C5'))

works.append(MusicalWork('random1', 'me', 2021, None, None, 'A4 B4 C5'))
works.append(MusicalWork('random2', 'me', 2021, None, None, 'C4 D4 E4'))
works.append(MusicalWork('random3', 'me', 2021, None, None, 'D4 D4 A4 A4 D5 A4 D4'))
works.append(MusicalWork('random4', 'me', 2021, None, None, 'Gb6 Bb6 Db7'))
works.append(MusicalWork('random5', 'me', 2021, None, None, 'F6 C6 A5 F5'))

In [4]:
class MW_Test:
    def __init__(self, title, melodyIntervals):
        self.title = title
        self.melodyIntervals = melodyIntervals
        
        self.searchCount = 0
    
    def __str__(self):
        return f'{self.title}: {self.melodyIntervals}'

In [5]:
testNames = ['base', 'short substring', 'long substring',
             'firstDiff = 4, longer', 'firstDiff = 2, longer', 'firstDiff = 2, shorter',
             'different', 'different2']

ivlsList = [[1,2,3,4,5,6], [1], [1,2,3,4],
            [1,2,3,7,8,9,8,7], [1,2,7,7,7,6,6,5], [1,2,7],
            [0,5,9,8], [0,5,8,7]]

# holds list of MW_Test objects containing above info
testWorks = []
for testName, ivls in zip(testNames, ivlsList):
    testWorks.append(MW_Test(testName, ivls))

# Testing the new parsing/valid function

In [6]:
def validMelody(melody):
    note = ''

    # go through every character, constructing a note and then checking it
    # extra space forces a check on the last note
    for i, char in enumerate(melody + ' '):
        print(i, char, note)
        
        # note construction
        if char == '/': note = ''
        elif char != ' ': note += char
        elif char == ' ' and note == '': continue
            
        # main case (a space with nonempty note): check for note, accidental, octave
        else:
            validNote = note[0].upper() in notes
            validAccidental = (len(note) == 2) or (note[1:len(note)-1].lower() in accidentals)
            validOctave = note[-1] in octavesStr

            # at least one is wrong
            if not (validNote and validAccidental and validOctave): return i - len(note)
            
            # otherwise, reset note
            print('all valid')
            note = ''

    # if we got here, entire melody was valid
    return -1

In [11]:
status = validMelody("Ab5      g B#5 Cx6")

print("Status:", status)

0 A 
1 b A
2 5 Ab
3   Ab5
all valid
4   
5   
6   
7   
8   
9 g 
10   g
Status: 9


# Compact Trie Testing for Find

In [5]:
ct = CompactTrie()

# insert works
for mw in works: ct.insert(mw)

In [7]:
print(ct.search('E6 D#6 E6 D#6 E6'))

[+2] Fur Elise | by Ludwig van Beethoven (1810, Classical Period)
Melody [a minor]: E5 D#5 E5 D#5 E5 B4 D5 C5 A4 / C4 E4 A4 B4 / E4 G#4 B4 C5


In [26]:
print(ct.search('G2 G2 D3 D3'))

[+5] random3 | by me (2021, None)
Melody [None]: D4 D4 A4 A4 D5 A4 D4


In [21]:
print(works[2])

[+3] Twinkle Twinkle Little Star | by None (None, None)
Melody [None]: C4 C4 G4 G4 A4 A4 G4


In [35]:
print(ct.root[0,7])

Intervals [7 0], with a nextArr
Terminal value of None
Most searched is random3 with 5 searches


# Compare Regular Trie and Compact Trie

## Regular Trie

In [5]:
tracemalloc.start()

In [6]:
t = Trie()

# insert works
for mw in testWorks: t.insert(mw)

In [7]:
regularCurrent, regularPeak = tracemalloc.get_traced_memory()
tracemalloc.stop()

## Compact Trie

In [8]:
tracemalloc.start()

In [9]:
ct = CompactTrie()

# insert works
for mw in testWorks: ct.insert(mw)

In [10]:
compactCurrent, compactPeak = tracemalloc.get_traced_memory()
tracemalloc.stop()

## Compare

In [11]:
factor, factorName = 10**3, 'kB'

print(f"Regular Trie: Current memory usage is {round(regularCurrent/factor, 3)}{factorName}; " +
      f"Peak was {round(regularPeak/factor, 3)}{factorName}")
print(f"Compact Trie: Current memory usage is {round(compactCurrent/factor, 3)}{factorName}; " +
      f"Peak was {round(compactPeak/factor, 3)}{factorName}")

Regular Trie: Current memory usage is 602.61kB; Peak was 655.37kB
Compact Trie: Current memory usage is 312.605kB; Peak was 365.541kB


# Compact Trie Testing for Insert

In [4]:
for i in range(len(testNames)):
    print(f'{i}:', ivlsList[i], '->', testNames[i])

0: [1, 2, 3, 4, 5, 6] -> base
1: [1] -> short substring
2: [1, 2, 3, 4] -> long substring
3: [1, 2, 3, 7, 8, 9, 8, 7] -> firstDiff = 4, longer
4: [1, 2, 7, 7, 7, 6, 6, 5] -> firstDiff = 2, longer
5: [1, 2, 7] -> firstDiff = 2, shorter
6: [0, 5, 9, 8] -> different
7: [0, 5, 8, 7] -> different2


In [5]:
testOrdersOriginal = np.arange(0,8)
testOrders = list(permutations(testOrdersOriginal))
len(testOrders)

40320

In [5]:
# DEFINE VALUES FOR MAIN TESTING BLOCK
nodesToCheck = [
    [1],
    [1,2],
    [1,2,3],
    [1,2,7],
    [1,2,3,4],
    [1,2,3,7],
    [1,2,7,7],
    [1,2,3,4,5],
    [0],
    [0,8],
    [0,9]
]

# (interval, terminal value)
nodeValues = [
    ([1], testNames[1]),
    ([2], None),
    ([3], None),
    ([7], testNames[5]),
    ([4], testNames[2]),
    ([7,8,9,8,7], testNames[3]),
    ([7,7,6,6,5], testNames[4]),
    ([5,6], testNames[0]),
    ([0,5], None),
    ([8,7], testNames[7]),
    ([9,8], testNames[6])
]

In [19]:
# MAIN TESTING BLOCK

def testTrie():
    print('Checking', end='')

    for orderNum, testOrder in enumerate(testOrders):
        if orderNum % 1000 == 0: print('.', end='')

        # Create trie and insert
        ct = CompactTrie()

        for i in testOrder:
            testName = testNames[i]
            ivls = ivlsList[i]

    #         print(f'\n[{i}] inserting', testName)
            ct.insert(MW_Test(testName, ivls))

        # Check validity of tree
        for nodeIvls, (correctIvls, correctTerminal) in zip(nodesToCheck, nodeValues):
            node = ct.root[nodeIvls]
            
            # 1. check intervals
            if len(node.intervals) != len(correctIvls) or \
               firstNonmatching(node.intervals, correctIvls) != -1:
                print(f'for order {orderNum} {testOrder}, node intervals {node.intervals} ' +
                      f'does not match with correct intervals {correctIvls}')
                return

            # 2. check terminal
            if node.terminalValue is None and correctTerminal is not None:
                print(f'for order {orderNum} {testOrder}, node terminal value is None when it ' +
                      f'should be "{correctTerminal}"')
                return
                
            elif node.terminalValue is not None and node.terminalValue.title != correctTerminal:
                print(f'for order {orderNum} {testOrder}, node terminal value "{node.terminalValue}" ' +
                      f'does not match with correct terminal value "{correctTerminal}"')
                return

            # 3. check that mostSearched is not None
            if node.mostSearched is None:
                print(f'for order {orderNum} {testOrder}, mostSearched of node {node.intervals} is None')

    # if we got here, success!
    print('Done')


testTrie()

Checking.........................................Done


# Regular Trie Testing

In [40]:
# START MALLOC
# https://medium.com/survata-engineering-blog/monitoring-memory-usage-of-a-running-python-program-49f027e3d1abmess
tracemalloc.start()

In [58]:
t = Trie()

In [ ]:
# insert
for mw in works: t.insert(mw)

In [60]:
print(t.search('G3 G3 A3 G3'))

[+1] Happy Birthday! | by None (None, None)
Melody [None]: C4 C4 D4 C4 F4 E4


In [61]:
print(t.search('Bb5 G6 F6 Eb6'))

[+1] Le Festin | by Ratatouille (2000, Movie Score)
Melody [Bb major]: Bb3 G4 F4 Eb4 G4 F4 Eb4 G4 F4 Eb4 Bb4


In [48]:
print(t.search('E5 D#5 E5 D#5 E5 B4 D5 C5 A4'))

[+1] Fur Elise | by Ludwig van Beethoven (1810, Classical Period)
Melody [a minor]: E5 D#5 E5 D#5 E5 B4 D5 C5 A4 / C4 E4 A4 B4 / E4 G#4 B4 C5


In [214]:
print(t.search('G4 F#4 G4 D4 C4'))

[+1] Jurrasic Park Theme | by John Williams (1993, Movie Score)
Melody [C major]: C5 B4 C5 G4 F4 C5 B4 C5 G4 F4 C5 B4 C5 D5 D5 F5 F5


In [215]:
print(t.search('Gd3 G3'))

[+1] random1 | by me (2021, None)
Melody [None]: A4 B4 C5


In [216]:
# STOP MALLOC
current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**3}kB; Peak was {peak / 10**3}kB")
tracemalloc.stop()

Current memory usage is 1591.625kB; Peak was 1648.448kB
